# **Études de cooccurrences de mots**
Auteur : PM Nugues

Une petite analyse des cooccurrences des racines *priv* et *publi[ck]* dans les débats du riksdag au XVI<sup>e</sup>, XVII<sup>e</sup> et XVIII<sup>e</sup> siècles. On peut appliquer ce programme à n'importe quelle séquence de caractères. Il suffit de changer quelques paramètres.

# Préliminaires

Les modules nécessaires

In [1]:
import operator
import regex as re
import sys
from math import log, sqrt
from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir

# Lecture des fichers XML et extraction des textes

## Liste des fichiers

On extrait la liste des fichers de débats. Les fichiers sont rassemblés pas décénnie.

In [2]:
dossiers = ['1520-40', '1540-60', '1560-80', '1580-1600','1600-1620', '1620-40', '1640-60', '1660-80', '1680-1700',
           '1700-1720', '1720-40']
dossiers = ['Riksdagstryck/' + dossier + '/' for dossier in dossiers]
dossiers

['Riksdagstryck/1520-40/',
 'Riksdagstryck/1540-60/',
 'Riksdagstryck/1560-80/',
 'Riksdagstryck/1580-1600/',
 'Riksdagstryck/1600-1620/',
 'Riksdagstryck/1620-40/',
 'Riksdagstryck/1640-60/',
 'Riksdagstryck/1660-80/',
 'Riksdagstryck/1680-1700/',
 'Riksdagstryck/1700-1720/',
 'Riksdagstryck/1720-40/']

In [3]:
fichiers_par_décénnie = [listdir(dossier) for dossier in dossiers]
fichiers_par_décénnie2 = []
fichiers_tous = []
for i in range(len(dossiers)):
    fichiers_par_décénnie2 += [[dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]]
    fichiers_tous += [dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]
fichiers_par_décénnie2

[['Riksdagstryck/1520-40/rda_1521-1560___01.xml'],
 ['Riksdagstryck/1540-60/rda_1521-1560___02.xml'],
 ['Riksdagstryck/1560-80/rda_1561-1592___01.xml'],
 ['Riksdagstryck/1580-1600/rda_1561-1592___02.xml',
  'Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml',
  'Riksdagstryck/1580-1600/rda_1597-1598___01.xml',
  'Riksdagstryck/1580-1600/rda_1597-1598___02.xml',
  'Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml'],
 ['Riksdagstryck/1600-1620/rda_1617_1_.xml',
  'Riksdagstryck/1600-1620/rda_1611-1616__.xml',
  'Riksdagstryck/1600-1620/rda_1617_2_.xml'],
 ['Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml',
  'Riksdagstryck/1620-40/roa_1627-1632__.xml',
  'Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml',
  'Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml'],
 ['Riksdagstryck/1640-60/roa_1660_1_.xml',
  'Riksdagstryck/1640-60/roa_1652-1655___02.xml',
  'Riksdagstryck/1640-60/roa_1652-1655___01.xml',
  'Riksdagstryck/1640-60/roa_1640-1644_2uppl_.xml',
  'Riksdagstryck/1640-60/pr_1642-1660___01.xml'

## Lecture des fichiers

On fait l'analyse syntaxique du code XML de chaque fichier

In [4]:
soups = []
for fichiers in fichiers_par_décénnie2:
    mini_soup = []
    for fichier in fichiers:
        print(fichier)
        with open(fichier, 'r', encoding='utf8') as infile:
            corpus_xml = infile.read()[1:]
        soup = BeautifulSoup(corpus_xml, 'xml')
        mini_soup += soup
    soups += [mini_soup]

Riksdagstryck/1520-40/rda_1521-1560___01.xml
Riksdagstryck/1540-60/rda_1521-1560___02.xml
Riksdagstryck/1560-80/rda_1561-1592___01.xml
Riksdagstryck/1580-1600/rda_1561-1592___02.xml
Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml
Riksdagstryck/1580-1600/rda_1597-1598___01.xml
Riksdagstryck/1580-1600/rda_1597-1598___02.xml
Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml
Riksdagstryck/1600-1620/rda_1617_1_.xml
Riksdagstryck/1600-1620/rda_1611-1616__.xml
Riksdagstryck/1600-1620/rda_1617_2_.xml
Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml
Riksdagstryck/1620-40/roa_1627-1632__.xml
Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml
Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml
Riksdagstryck/1640-60/roa_1660_1_.xml
Riksdagstryck/1640-60/roa_1652-1655___02.xml
Riksdagstryck/1640-60/roa_1652-1655___01.xml
Riksdagstryck/1640-60/roa_1640-1644_2uppl_.xml
Riksdagstryck/1640-60/pr_1642-1660___01.xml
Riksdagstryck/1640-60/roa_1645-1651__.xml
Riksdagstryck/1640-60/pr_1642-1660___02.xml
Riksdagstryck/1640

On vérifie le nombre de décénnies

In [5]:
len(soups)

11

On concatène les lignes de chaque fichier de sorte qu'il n'y ait qu'une ligne de texte par fichier

In [6]:
textes = []
cnt = 0
print('Nom du fichier et nombre de lignes')
for mini_soups in soups:
    textes_par_décénnie = []
    for soup in mini_soups:
        lines = soup.find_all('line')
        print(fichiers_tous[cnt], ':', len(lines))
        cnt += 1
        texte = ''
        for line in lines:
            texte = texte + line.get_text() + '\n'
        #textes_par_décénnie += texte
        textes_par_décénnie += [texte]
    textes += [textes_par_décénnie]

Nom du fichier et nombre de lignes
Riksdagstryck/1520-40/rda_1521-1560___01.xml : 27197
Riksdagstryck/1540-60/rda_1521-1560___02.xml : 12517
Riksdagstryck/1560-80/rda_1561-1592___01.xml : 29639
Riksdagstryck/1580-1600/rda_1561-1592___02.xml : 28356
Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml : 25162
Riksdagstryck/1580-1600/rda_1597-1598___01.xml : 28272
Riksdagstryck/1580-1600/rda_1597-1598___02.xml : 19203
Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml : 27741
Riksdagstryck/1600-1620/rda_1617_1_.xml : 11009
Riksdagstryck/1600-1620/rda_1611-1616__.xml : 25737
Riksdagstryck/1600-1620/rda_1617_2_.xml : 6102
Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml : 15164
Riksdagstryck/1620-40/roa_1627-1632__.xml : 11108
Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml : 12337
Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml : 9640
Riksdagstryck/1640-60/roa_1660_1_.xml : 10276
Riksdagstryck/1640-60/roa_1652-1655___02.xml : 5791
Riksdagstryck/1640-60/roa_1652-1655___01.xml : 24470
Riksdagstryck/1640-

On vérifie le nombre de textes

In [7]:
len(textes)

11

Nombre de textes par décénnie

In [8]:
list(map(len, textes))

[1, 1, 1, 5, 3, 4, 9, 9, 5, 9, 27]

# Concordances

Les séquences de caractères à rechercher et le contexte de la concordance

On fixe le choix à *priv* ou *publi(c|k|que)*. On peut les remplacer par n'importe quelle séquence

In [9]:
PRIVAT = False

In [10]:
if PRIVAT:
    chaîne = 'privat'
else:
    chaîne = 'publi(k|c|que)'
contexte = 25

In [11]:
chaîne = re.sub(' ', '\\s+', chaîne)

In [12]:
for i, décénnie in enumerate(textes):
    print('===DÉCÉNNIE===', i + 1)
    for i, texte in enumerate(décénnie):
        print(fichiers[i], ':')
        texte = re.sub('\s+', ' ', texte.lower())
        concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
                       .format(pattern=chaîne, width=contexte))
        for match in re.finditer(concordance, texte):
            print('  ', match.group(1))

===DÉCÉNNIE=== 1
Riksdagstryck/1720-40/pr_1723___02.xml :
   entlighet; den äldsta så publicerade urkunden är det ryk
   n e. hildebrand, svenska publikationer af historiska han
   ing, och nyttan af dessa publikationer blef därigenom vä
    började den regelbundna publikationen af samtliga ständ
   n senare påbörjad större publikation af ståndens protoko
   med stats¬ medel utgifna publikationer öfverlemnades åt 
   , för att förverkliga en publikation, som så länge hade 
    riksarkivets historiska publikationer. denna anordning 
   tadelfri samling af acta publica från gustaf i:s och föl
   iskt bibliotek 1878. den publikation, som här föreligger
   te anses att i en urkundspublikation öfversätta en handl
   n större svensk urkunds- publikation tillämpa modifikati
   on- stitucionem. 6. item publica penitencia im- ponenda 
   nstitutiones svetica, in publicis comitiis regnorum svec
   ulgari sermone postmodum public atos, in lingnarn latina
   cti et de aliis negociis publicis tract

# Cooccurrences

## Mots vides

On définit une liste de mots vides. On ne l'utilise pas dans le reste du programme

In [13]:
mots_vides = ['och', 'den', 'också', 'för']

## Fonctions de comptage des mots et des bigrammes

On définit les fonctions de comptage. Elles sont tirées de l'édition à venir de mon livre.

In [14]:
def découpe_mots(texte):
    mots = re.findall("\p{L}+|\p{N}+", texte)
    return mots


"""def découpe_mots(texte):
    mots = re.split('[\s\-,;:!?.’\'«»()–...&‘’“”*—]+', texte)
    mots.remove('')
    return mots"""


def compte_mots(mots):
    c_mots = {}
    for mot in mots:
        c_mots[mot] = c_mots.get(mot, 0) + 1
    return c_mots


def compte_bigrammes(mots):
    c_bigrammes = {}
    for i in range(len(mots) - 1):
        bigramme = (mots[i], mots[i + 1])
        c_bigrammes[bigramme] = c_bigrammes.get(bigramme, 0) + 1
    return c_bigrammes


def compte_bigrammes_fen(mots, fen=5):
    c_bigrammes_fen = {}
    for inx in range(1, fen + 1):
        for i in range(len(mots) - inx):
            bigramme = (mots[i], mots[i + inx])
            c_bigrammes_fen[bigramme] = c_bigrammes_fen.get(bigramme, 0) + 1
    return c_bigrammes_fen

On concatène les textes par décénnie. Pour chaque décénnie, on aura donc une ligne de texte

In [15]:
textes = [' '.join(décénnie) for décénnie in textes]

On crée aussi une chaîne avec le corpus en entier

In [16]:
corpus_entier = ' '.join(textes)

## Les mots

On compte les mots par décénnie

In [17]:
c_mots_ordonnés_par_corpus = []
c_mots_par_corpus = []
textes_découpés = []
total_mots_par_corpus = []
for i, texte in enumerate(textes):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    texte = texte.lower()
    texte_découpé = découpe_mots(texte)
    total_mots_par_corpus += [len(texte_découpé)]
    textes_découpés += [texte_découpé]
    # print(mots)
    c_mots = compte_mots(texte_découpé)
    c_mots_par_corpus += [c_mots]
    c_mots_ordonnés = sorted(c_mots.items(), key=operator.itemgetter(1), reverse=True)
    c_mots_ordonnés_par_corpus += [c_mots_ordonnés]
    print('  ', c_mots_ordonnés[:10])
print('Nombre total de mots:', total_mots_par_corpus)

===DÉCÉNNIE=== 1
   [('och', 13902), ('i', 5621), ('som', 3428), ('till', 2798), ('att', 2643), ('the', 2380), ('af', 1982), ('så', 1835), ('ther', 1780), ('eller', 1777)]
===DÉCÉNNIE=== 2
   [('och', 6741), ('i', 2277), ('till', 1908), ('som', 1459), ('att', 1272), ('af', 1218), ('för', 869), ('så', 860), ('vi', 841), ('på', 829)]
===DÉCÉNNIE=== 3
   [('och', 12809), ('i', 6603), ('till', 3317), ('som', 3238), ('att', 3191), ('the', 2176), ('af', 2130), ('så', 1984), ('vi', 1710), ('för', 1664)]
===DÉCÉNNIE=== 4
   [('och', 65349), ('i', 24590), ('till', 17900), ('att', 16727), ('som', 16420), ('h', 12905), ('k', 12051), ('m', 10188), ('n', 9850), ('så', 9496)]
===DÉCÉNNIE=== 5
   [('och', 24517), ('i', 7100), ('att', 6696), ('till', 6019), ('som', 5668), ('m', 5029), ('h', 4675), ('k', 4380), ('t', 4137), ('så', 3631)]
===DÉCÉNNIE=== 6
   [('och', 19711), ('att', 9516), ('till', 6020), ('som', 4446), ('dhe', 4305), ('i', 4174), ('den', 3646), ('af', 3354), ('på', 3345), ('för', 2851)

Pour tout le corpus

In [18]:
corpus_entier = corpus_entier.lower()
corpus_découpé = découpe_mots(corpus_entier)
corpus_mots = compte_mots(corpus_découpé)
corpus_mots_ordonnés = sorted(corpus_mots.items(), key=operator.itemgetter(1), reverse=True)
print('  ', corpus_mots_ordonnés[:10])

   [('och', 395754), ('i', 159607), ('som', 141755), ('at', 121270), ('att', 114557), ('till', 99313), ('af', 95763), ('så', 85387), ('den', 77590), ('för', 74976)]


On filtre les mots qui contiennent la séquence de caractères par décénnie

In [19]:
occurrences_chaîne = []
for i, c_mots_ordonnés in enumerate(c_mots_ordonnés_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    #print(fichiers[i], ':')
    occurrences_chaîne_1 = list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés))
    occurrences_chaîne += [occurrences_chaîne_1]
    print(' ', list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés)))

===DÉCÉNNIE=== 1
  [('publikationer', 4), ('publikation', 4), ('publica', 3), ('publicis', 2), ('publicerade', 1), ('publikationen', 1), ('urkundspublikation', 1), ('public', 1), ('publiceras', 1)]
===DÉCÉNNIE=== 2
  [('publica', 2), ('publicera', 1), ('publikationerna', 1)]
===DÉCÉNNIE=== 3
  [('publikation', 3), ('publiceres', 3), ('publica', 3), ('publicere', 3), ('publicis', 2), ('publiceret', 1), ('publicam', 1), ('publicus', 1), ('publicas', 1), ('publice', 1), ('publicum', 1)]
===DÉCÉNNIE=== 4
  [('publica', 14), ('publice', 14), ('reipublicae', 7), ('publicis', 5), ('publico', 5), ('publicum', 4), ('publiciren', 4), ('publiciret', 4), ('publicera', 4), ('publiceret', 3), ('publikation', 3), ('publicerandet', 3), ('publicering', 3), ('publicam', 3), ('publikationer', 2), ('publication', 2), ('publicerat', 2), ('publicus', 2), ('public', 1), ('publicere', 1), ('publiceradt', 1), ('publiceras', 1), ('publicieret', 1), ('publicisque', 1), ('aktpublikatio', 1), ('publicerades', 1), 

Pour tout le corpus

In [20]:
list(filter(lambda x: re.search(chaîne, x[0]), corpus_mots_ordonnés))

[('publicum', 411),
 ('publique', 350),
 ('publico', 344),
 ('publici', 320),
 ('publica', 212),
 ('publication', 113),
 ('publice', 112),
 ('publiceras', 97),
 ('publicera', 62),
 ('publicerade', 61),
 ('publicerat', 60),
 ('publika', 50),
 ('publicam', 38),
 ('publicerad', 38),
 ('publikation', 37),
 ('republiquen', 35),
 ('publicis', 32),
 ('public', 30),
 ('publicationen', 28),
 ('republiquer', 24),
 ('publicerande', 22),
 ('publicus', 19),
 ('publicationer', 16),
 ('publikationen', 15),
 ('publikationer', 14),
 ('publicerades', 13),
 ('republique', 13),
 ('reipublicae', 12),
 ('publicos', 11),
 ('reipublicse', 10),
 ('publikt', 10),
 ('publicse', 9),
 ('publiques', 9),
 ('republiquens', 9),
 ('publiquement', 9),
 ('publiceres', 8),
 ('publicere', 7),
 ('publiceret', 7),
 ('publicering', 7),
 ('republiken', 7),
 ('publiceradt', 6),
 ('republiquers', 6),
 ('respubliquer', 6),
 ('republiker', 6),
 ('publiquet', 6),
 ('reipublic', 5),
 ('respubliker', 5),
 ('publicé', 5),
 ('publict',

### Fréquence absolue de la séquence

In [21]:
occurrences_chaîne

[[('publikationer', 4),
  ('publikation', 4),
  ('publica', 3),
  ('publicis', 2),
  ('publicerade', 1),
  ('publikationen', 1),
  ('urkundspublikation', 1),
  ('public', 1),
  ('publiceras', 1)],
 [('publica', 2), ('publicera', 1), ('publikationerna', 1)],
 [('publikation', 3),
  ('publiceres', 3),
  ('publica', 3),
  ('publicere', 3),
  ('publicis', 2),
  ('publiceret', 1),
  ('publicam', 1),
  ('publicus', 1),
  ('publicas', 1),
  ('publice', 1),
  ('publicum', 1)],
 [('publica', 14),
  ('publice', 14),
  ('reipublicae', 7),
  ('publicis', 5),
  ('publico', 5),
  ('publicum', 4),
  ('publiciren', 4),
  ('publiciret', 4),
  ('publicera', 4),
  ('publiceret', 3),
  ('publikation', 3),
  ('publicerandet', 3),
  ('publicering', 3),
  ('publicam', 3),
  ('publikationer', 2),
  ('publication', 2),
  ('publicerat', 2),
  ('publicus', 2),
  ('public', 1),
  ('publicere', 1),
  ('publiceradt', 1),
  ('publiceras', 1),
  ('publicieret', 1),
  ('publicisque', 1),
  ('aktpublikatio', 1),
  ('pu

### Fréquence relative de la séquence par décénnie

In [22]:
for i in range(len(total_mots_par_corpus)):
    print(sum([x[1] for x in occurrences_chaîne[i]])/total_mots_par_corpus[i])

7.258035249857864e-05
3.572704537334762e-05
7.969175230209549e-05
8.129890506299408e-05
0.00019042107788222085
7.701723260579554e-05
0.0002718936627714389
0.00037125281832263516
0.00035179906610218647
0.00039222565845265444
0.0003697821847093708


## Les bigrammes

### Les bigrammes comme mots adjacents

On compte les bigrammes et on les ordonne par fréquence pour chaque décénnie

In [23]:
c_bigrammes_ordonnés_par_corpus = []
c_bigrammes_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    print('===DÉCÉNNIE===', i + 1)
    # print(fichiers[i], ':')
    c_bigrammes = compte_bigrammes(texte_découpé)
    c_bigrammes_par_corpus += [c_bigrammes]
    c_bigrammes_ordonnés = sorted(c_bigrammes.items(), 
                                  key=operator.itemgetter(1), reverse=True)
    c_bigrammes_ordonnés_par_corpus += [c_bigrammes_ordonnés]
    print('  ', c_bigrammes_ordonnés[:15])

===DÉCÉNNIE=== 1
   [(('k', 'm'), 475), (('m', 't'), 450), (('hans', 'nåde'), 329), (('h', 'k'), 304), (('ma', 't'), 271), (('att', 'the'), 235), (('och', 'så'), 231), (('i', 's'), 191), (('hans', 'nade'), 183), (('som', 'i'), 174), (('som', 'the'), 171), (('ath', 'the'), 165), (('och', 'en'), 160), (('hvar', 'och'), 152), (('och', 'i'), 150)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 227), (('m', 't'), 149), (('k', 'gustafs'), 143), (('bref', 'till'), 142), (('ma', 't'), 133), (('kong', 'e'), 125), (('till', 'att'), 118), (('gustafs', 'bref'), 112), (('h', 'k'), 109), (('och', 'så'), 106), (('vi', 'och'), 106), (('som', 'vi'), 98), (('så', 'att'), 95), (('och', 'hans'), 94), (('högbe', 'te'), 93)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 553), (('h', 'k'), 428), (('kong', 'e'), 408), (('m', 't'), 345), (('att', 'the'), 333), (('till', 'att'), 271), (('e', 'k'), 251), (('ma', 't'), 219), (('e', 'ma'), 212), (('ther', 'till'), 207), (('att', 'vi'), 186), (('m', 'tz'), 184), (('så', 'och'), 181), (('m'

Et pour le corpus en entier

In [24]:
corpus_bigrammes = compte_bigrammes(corpus_découpé)
corpus_bigrammes_ordonnés = sorted(corpus_bigrammes.items(), 
                              key=operator.itemgetter(1), reverse=True)
corpus_bigrammes_ordonnés[:15]

[(('k', 'm'), 22400),
 (('kongl', 'maij'), 15905),
 (('h', 'k'), 12249),
 (('maij', 't'), 12108),
 (('m', 't'), 11884),
 (('h', 'r'), 11699),
 (('och', 'adelen'), 9001),
 (('f', 'n'), 8291),
 (('r', 'o'), 7913),
 (('o', 'a'), 7444),
 (('ridderskapet', 'och'), 7103),
 (('h', 'f'), 6110),
 (('hans', 'kongl'), 6109),
 (('m', 'tz'), 6078),
 (('at', 'de'), 5853)]

Les bigrammes qui contienne la séquence de caractères pour chaque décénnie

In [25]:
for i, c_bigrammes_ordonnés in enumerate(c_bigrammes_ordonnés_par_corpus):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(' ', list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1]), 
                      c_bigrammes_ordonnés)))

===DÉCÉNNIE=== 1
  [(('publikation', 'som'), 2), (('acta', 'publica'), 2), (('publica', 'från'), 2), (('så', 'publicerade'), 1), (('publicerade', 'urkunden'), 1), (('svenska', 'publikationer'), 1), (('publikationer', 'af'), 1), (('dessa', 'publikationer'), 1), (('publikationer', 'blef'), 1), (('regelbundna', 'publikationen'), 1), (('publikationen', 'af'), 1), (('större', 'publikation'), 1), (('publikation', 'af'), 1), (('utgifna', 'publikationer'), 1), (('publikationer', 'öfverlemnades'), 1), (('en', 'publikation'), 1), (('historiska', 'publikationer'), 1), (('publikationer', 'denna'), 1), (('den', 'publikation'), 1), (('en', 'urkundspublikation'), 1), (('urkundspublikation', 'öfversätta'), 1), (('urkunds', 'publikation'), 1), (('publikation', 'tillämpa'), 1), (('item', 'publica'), 1), (('publica', 'penitencia'), 1), (('in', 'publicis'), 1), (('publicis', 'comitiis'), 1), (('postmodum', 'public'), 1), (('public', 'atos'), 1), (('negociis', 'publicis'), 1), (('publicis', 'tractabitur'),

Et pour le corpus en entier. On ne retient que les couples avec une fréquence supérieure à 5

In [26]:
list(filter(lambda x: (re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1])) and x[1] >= 5, 
                      corpus_bigrammes_ordonnés))

[(('til', 'publici'), 73),
 (('af', 'publico'), 68),
 (('för', 'publico'), 60),
 (('och', 'publici'), 51),
 (('publici', 'nytta'), 48),
 (('acta', 'publica'), 39),
 (('till', 'publici'), 39),
 (('til', 'publicum'), 37),
 (('de', 'publique'), 36),
 (('och', 'publicum'), 34),
 (('för', 'publici'), 34),
 (('publico', 'till'), 33),
 (('publicum', 'och'), 29),
 (('en', 'publication'), 29),
 (('publico', 'och'), 25),
 (('till', 'publicum'), 25),
 (('fides', 'publica'), 25),
 (('publica', 'och'), 24),
 (('at', 'publicum'), 24),
 (('som', 'publicum'), 24),
 (('publici', 'bästa'), 23),
 (('andre', 'publique'), 22),
 (('publique', 'medel'), 22),
 (('och', 'publico'), 21),
 (('salus', 'publica'), 18),
 (('publici', 'tienst'), 18),
 (('publici', 'tiänst'), 18),
 (('och', 'republiquer'), 16),
 (('publique', 'och'), 16),
 (('publico', 'til'), 16),
 (('af', 'publique'), 16),
 (('publicerat', 'och'), 15),
 (('fide', 'publica'), 15),
 (('jus', 'publicum'), 15),
 (('publiceras', 'och'), 14),
 (('och', '

### Les bigrammes définis dans une fenêtre

Les bigrammes avec une fenêtre. On considère maintenant le contexte à droite et à gauche du mot avec une fenêtre de 5 mots

Les bigrammes les plus fréquents pour chaque décénnie

In [27]:
c_bigrammes_fen_ordonnés_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    c_bigrammes_fen = compte_bigrammes_fen(texte_découpé)
    c_bigrammes_fen_ordonnés = sorted(c_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
    c_bigrammes_fen_ordonnés_par_corpus += [c_bigrammes_fen_ordonnés]
    print(c_bigrammes_fen_ordonnés[:10])

===DÉCÉNNIE=== 1
[(('och', 'och'), 3309), (('och', 'i'), 1285), (('och', 'som'), 1135), (('i', 'och'), 1110), (('och', 'att'), 955), (('som', 'och'), 810), (('till', 'och'), 798), (('och', 'så'), 775), (('och', 'the'), 755), (('och', 'till'), 744)]
===DÉCÉNNIE=== 2
[(('och', 'och'), 1616), (('och', 'som'), 560), (('till', 'och'), 527), (('och', 'till'), 470), (('och', 'så'), 410), (('som', 'och'), 396), (('och', 'i'), 391), (('och', 'att'), 382), (('i', 'och'), 363), (('och', 'ther'), 351)]
===DÉCÉNNIE=== 3
[(('och', 'och'), 2802), (('i', 'i'), 2256), (('och', 'som'), 1015), (('och', 'att'), 989), (('och', 'i'), 946), (('i', 'och'), 846), (('och', 'till'), 842), (('till', 'och'), 828), (('och', 'så'), 780), (('som', 'och'), 722)]
===DÉCÉNNIE=== 4
[(('och', 'och'), 14958), (('k', 'm'), 8598), (('f', 'n'), 7322), (('och', 'i'), 5818), (('h', 'f'), 5461), (('till', 'och'), 5401), (('och', 'som'), 5346), (('och', 'att'), 5264), (('h', 'n'), 5132), (('h', 'k'), 5056)]
===DÉCÉNNIE=== 5
[(('o

Et pour le corpus entier

In [28]:
corpus_bigrammes_fen = compte_bigrammes_fen(corpus_découpé)
corpus_bigrammes_ordonnés_fen = sorted(corpus_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
corpus_bigrammes_ordonnés_fen[:15]

[(('och', 'och'), 65469),
 (('och', 'som'), 30633),
 (('och', 'i'), 30558),
 (('och', 'att'), 28273),
 (('till', 'och'), 25904),
 (('i', 'och'), 25620),
 (('som', 'och'), 23630),
 (('och', 'till'), 23214),
 (('k', 'm'), 22843),
 (('och', 'så'), 20030),
 (('och', 'at'), 19199),
 (('att', 'och'), 18467),
 (('och', 'af'), 18087),
 (('af', 'och'), 17361),
 (('och', 'för'), 16518)]

Les bigrammes avec la séquence de caractères recherchée par décénnie

In [29]:
for i, c_bigrammes_fen_ordonnés in enumerate(c_bigrammes_fen_ordonnés_par_corpus):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                      re.search(chaîne, x[0][1]), 
                      c_bigrammes_fen_ordonnés)))

===DÉCÉNNIE=== 1
[(('en', 'publikation'), 3), (('dessa', 'publikationer'), 2), (('större', 'publikation'), 2), (('publikation', 'af'), 2), (('publikation', 'som'), 2), (('acta', 'publica'), 2), (('publica', 'från'), 2), (('af', 'publikationer'), 2), (('af', 'publica'), 2), (('i', 'publikation'), 2), (('så', 'publicerade'), 1), (('publicerade', 'urkunden'), 1), (('svenska', 'publikationer'), 1), (('publikationer', 'af'), 1), (('publikationer', 'blef'), 1), (('regelbundna', 'publikationen'), 1), (('publikationen', 'af'), 1), (('utgifna', 'publikationer'), 1), (('publikationer', 'öfverlemnades'), 1), (('historiska', 'publikationer'), 1), (('publikationer', 'denna'), 1), (('den', 'publikation'), 1), (('en', 'urkundspublikation'), 1), (('urkundspublikation', 'öfversätta'), 1), (('urkunds', 'publikation'), 1), (('publikation', 'tillämpa'), 1), (('item', 'publica'), 1), (('publica', 'penitencia'), 1), (('in', 'publicis'), 1), (('publicis', 'comitiis'), 1), (('postmodum', 'public'), 1), (('pub

Et pour le corpus en entier

In [30]:
list(filter(lambda x: (re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1])) and x[1] >= 5, 
                      corpus_bigrammes_ordonnés_fen))

[(('och', 'publici'), 120),
 (('publique', 'och'), 110),
 (('publici', 'och'), 104),
 (('publico', 'och'), 95),
 (('och', 'publicum'), 85),
 (('publicum', 'och'), 84),
 (('och', 'publique'), 84),
 (('til', 'publici'), 84),
 (('af', 'publico'), 82),
 (('för', 'publico'), 72),
 (('publici', 'nytta'), 69),
 (('till', 'publici'), 64),
 (('publica', 'och'), 61),
 (('de', 'publique'), 57),
 (('publico', 'till'), 55),
 (('och', 'publico'), 55),
 (('som', 'publicum'), 54),
 (('at', 'publicum'), 52),
 (('en', 'publication'), 48),
 (('til', 'publicum'), 46),
 (('publique', 'som'), 46),
 (('för', 'publici'), 44),
 (('at', 'publique'), 43),
 (('i', 'publique'), 40),
 (('acta', 'publica'), 39),
 (('som', 'publico'), 38),
 (('som', 'publique'), 38),
 (('det', 'publicum'), 38),
 (('som', 'publici'), 38),
 (('till', 'publicum'), 37),
 (('af', 'publique'), 35),
 (('at', 'publico'), 34),
 (('till', 'publique'), 33),
 (('publico', 'som'), 33),
 (('publicum', 'at'), 33),
 (('publicum', 'i'), 32),
 (('publ

# Mesures de cooccurrence

On applique maintenant trois mesures de cooccurrence :
* l'information mutuelle de Fano
* les t-scores
* le rapport de vraisemblance

## Information mutuelle

Définition de l'information mutuelle

In [31]:
def info_mutuelle(c_mots, c_bigrammes, taille):
    c_info_mutuelle = {}
    for bigramme in c_bigrammes.keys():
        c_info_mutuelle[bigramme] = log(taille * c_bigrammes[bigramme] / 
                                        (c_mots[bigramme[0]] * c_mots[bigramme[1]]), 2.0)
    return c_info_mutuelle

On ordonne les bigrammes par valeur d'information mutuelle décroissante

In [32]:
c_info_mutuelle_ordonnée_par_corpus = []
c_info_mutuelle_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_info_mutuelle = info_mutuelle(c_mots, c_bigrammes, len(texte_découpé))
    c_info_mutuelle_par_corpus += [c_info_mutuelle]
    c_info_mutuelle_ordonnée = sorted(c_info_mutuelle.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_info_mutuelle_ordonnée[:10])
    c_info_mutuelle_ordonnée_par_corpus += [c_info_mutuelle_ordonnée]

===DÉCÉNNIE=== 1
   [(('ehva', 'rfv'), 17.91998641235595), (('konseqvent', 'anlita'), 17.91998641235595), (('anlita', 'tryckpressen'), 17.91998641235595), (('lagens', 'revision'), 17.91998641235595), (('liturgien', '1577'), 17.91998641235595), (('confessio', 'fidei'), 17.91998641235595), (('fidei', '1593'), 17.91998641235595), (('1726', '4727'), 17.91998641235595), (('1728', '1733'), 17.91998641235595), (('1733', 'utgaf'), 17.91998641235595)]
===DÉCÉNNIE=== 2
   [(('kop', 'perbergs'), 16.77262386646028), (('listeligen', 'trachtede'), 16.77262386646028), (('bätter', 'tilförlatt'), 16.77262386646028), (('olämpe', 'tilskvnde'), 16.77262386646028), (('forrätlige', 'hiärter'), 16.77262386646028), (('hiärter', 'ingifne'), 16.77262386646028), (('ohörsamhet', 'gemenligen'), 16.77262386646028), (('unsacht', 'hotedt'), 16.77262386646028), (('pfaltzgref', 'viske'), 16.77262386646028), (('almänneligen', 'rychtes'), 16.77262386646028)]
===DÉCÉNNIE=== 3
   [(('tillsvidare', 'afbryta'), 17.9371381491

L'information mutuelle des mots plus fréquents avec un seuil. Le seuil est assez haut pour le pas avoir trop d'affichages

In [33]:
seuil = 200
for i, c_mots, c_bigrammes, c_info_mutuelle, c_info_mutuelle_ordonnée in zip(
    range(len(textes)),
    c_mots_par_corpus,
    c_bigrammes_par_corpus,
    c_info_mutuelle_par_corpus,
    c_info_mutuelle_ordonnée_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    for bigramme_im in c_info_mutuelle_ordonnée:
        if c_bigrammes[bigramme_im[0]] >= seuil:
            #mots_du_bigramme = bigramme_im[0].split()
            print('  ', c_info_mutuelle[bigramme_im[0]], "\t", 
                  bigramme_im[0], "\t", c_bigrammes[bigramme_im[0]], "\t",
                  c_mots[bigramme_im[0][0]], "\t", c_mots[bigramme_im[0][1]])

===DÉCÉNNIE=== 1
   7.821400192633897 	 ('h', 'k') 	 304 	 434 	 768
   7.651268368018571 	 ('k', 'm') 	 475 	 768 	 763
   7.5664977449239235 	 ('ma', 't') 	 271 	 408 	 869
   7.395015989924054 	 ('m', 't') 	 450 	 763 	 869
   7.004215984001913 	 ('hans', 'nåde') 	 329 	 1485 	 428
   3.2117967910840712 	 ('att', 'the') 	 235 	 2643 	 2380
   1.1671662764554518 	 ('och', 'så') 	 231 	 13902 	 1835
===DÉCÉNNIE=== 2
   6.294201680414793 	 ('k', 'm') 	 227 	 649 	 499
===DÉCÉNNIE=== 3
   7.830036487578805 	 ('kong', 'e') 	 408 	 462 	 974
   7.475185796216387 	 ('e', 'ma') 	 212 	 974 	 307
   7.324416892853389 	 ('ma', 't') 	 219 	 307 	 1117
   6.704669513043162 	 ('k', 'm') 	 553 	 1154 	 1153
   6.232382027647109 	 ('h', 'k') 	 428 	 1238 	 1154
   6.071000432636152 	 ('m', 't') 	 345 	 1153 	 1117
   5.8084762323286565 	 ('e', 'k') 	 251 	 974 	 1154
   3.589260781647319 	 ('att', 'the') 	 333 	 3191 	 2176
   3.3257830716750325 	 ('ther', 'till') 	 207 	 1562 	 3317
   2.68383100

Les bigrammes avec la séquence qu'on recherche ordonnés par information mutuelle 

In [34]:
for i, corpus in enumerate(c_info_mutuelle_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('regelbundna', 'publikationen'), 17.91998641235595), (('urkundspublikation', 'öfversätta'), 17.91998641235595), (('postmodum', 'public'), 17.91998641235595), (('public', 'atos'), 17.91998641235595), (('publicerade', 'urkunden'), 16.91998641235595), (('negociis', 'publicis'), 16.91998641235595), (('publicis', 'tractabitur'), 16.91998641235595), (('publica', 'penitencia'), 16.335023911634792), (('publikationer', 'öfverlemnades'), 15.919986412355948), (('urkunds', 'publikation'), 15.919986412355948), (('publikation', 'tillämpa'), 15.919986412355948), (('publicis', 'comitiis'), 15.919986412355948), (('utgifna', 'publikationer'), 14.33502391163479), (('acta', 'publica'), 13.6345841934937), (('historiska', 'publikationer'), 12.598058317468585), (('större', 'publikation'), 10.790703395410981), (('dessa', 'publikationer'), 10.16509891019248), (('svenska', 'publikationer'), 9.710533046726997), (('publica', 'från'), 9.290629792276338), (('skulle', 'publiceras'), 8.96870169738

## t-scores

Définition des t-scores

In [35]:
def t_scores(words, freq_unigrams, freq_bigrams):
    ts = {}
    for bigram in freq_bigrams:
        ts[bigram] = ((freq_bigrams[bigram] -
                      freq_unigrams[bigram[0]] *
                      freq_unigrams[bigram[1]] /
                      len(words)) /
                      sqrt(freq_bigrams[bigram]))
    return ts

On calcule les t-scores et on affiche les plus hauts

In [36]:
c_t_scores_ordonnée_par_corpus = []
c_t_scores_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    # print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_t_scores = t_scores(texte_découpé, c_mots, c_bigrammes)
    c_t_scores_par_corpus += [c_t_scores]
    c_t_scores_ordonnée = sorted(c_t_scores.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_t_scores_ordonnée[:10])
    c_t_scores_ordonnée_par_corpus += [c_t_scores_ordonnée]

===DÉCÉNNIE=== 1
   [(('k', 'm'), 21.686080679537064), (('m', 't'), 21.08717033359404), (('hans', 'nåde'), 17.997064734352747), (('h', 'k'), 17.358512398402247), (('ma', 't'), 16.375233224539944), (('att', 'the'), 13.675133577526216), (('hans', 'nade'), 13.430369171605783), (('i', 's'), 12.854315802699043), (('ath', 'the'), 11.994280160000711), (('gode', 'men'), 11.587086450551894)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 14.874533233532865), (('m', 't'), 12.03312006199931), (('k', 'gustafs'), 11.879247226209095), (('bref', 'till'), 11.488770561609524), (('ma', 't'), 11.462297721515135), (('kong', 'e'), 11.141686828183712), (('gustafs', 'bref'), 10.541872575177774), (('h', 'k'), 10.332592971694531), (('högbe', 'te'), 9.625890323606253), (('hans', 'kong'), 9.420878525876393)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 23.290499077995012), (('h', 'k'), 20.412999187239397), (('kong', 'e'), 20.110242327092337), (('m', 't'), 18.297891252925407), (('att', 'the'), 16.732120872246025), (('e', 'k'), 15.560288

Les bigrammes avec la séquence qu'on recherche ordonnés par t-scores 

In [37]:
for i, corpus in enumerate(c_t_scores_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('acta', 'publica'), 1.4141023645776576), (('publica', 'från'), 1.411955391758058), (('publikation', 'som'), 1.3751175579875081), (('regelbundna', 'publikationen'), 0.9999959677581945), (('urkundspublikation', 'öfversätta'), 0.9999959677581945), (('postmodum', 'public'), 0.9999959677581945), (('public', 'atos'), 0.9999959677581945), (('publicerade', 'urkunden'), 0.9999919355163891), (('negociis', 'publicis'), 0.9999919355163891), (('publicis', 'tractabitur'), 0.9999919355163891), (('publica', 'penitencia'), 0.9999879032745835), (('publikationer', 'öfverlemnades'), 0.9999838710327781), (('urkunds', 'publikation'), 0.9999838710327781), (('publikation', 'tillämpa'), 0.9999838710327781), (('publicis', 'comitiis'), 0.9999838710327781), (('utgifna', 'publikationer'), 0.9999516130983342), (('historiska', 'publikationer'), 0.9998387103277809), (('större', 'publikation'), 0.9994354861472333), (('dessa', 'publikationer'), 0.9991290357700171), (('svenska', 'publikationer'), 0.9

## Log du rapport de vraisemblance (log-likelihood-ration)

Définition du rapport de vraisemblance

In [38]:
def likelihood_ratio(words, freq_unigrams, freq_bigrams):
    lr = {}
    for bigram in freq_bigrams:
        p = freq_unigrams[bigram[1]] / len(words)
        p1 = freq_bigrams[bigram] / freq_unigrams[bigram[0]]
        p2 = ((freq_unigrams[bigram[1]] - freq_bigrams[bigram])
              / (len(words) - freq_unigrams[bigram[0]]))
        if p1 != 1.0 and p2 != 0.0:
            lr[bigram] = 2.0 * (
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p1) +
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p2) -
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p) -
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p))
    return lr

In [39]:
def log_f(k, N, p):
    return k * log(p) + (N - k) * log(1 - p)

On le calcule

In [40]:
c_lr_ordonné_par_corpus = []
c_lr_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_lr = likelihood_ratio(texte_découpé, c_mots, c_bigrammes)
    c_lr_par_corpus += [c_lr]
    c_lr_ordonné = sorted(c_lr.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_lr_ordonné[:10])
    c_lr_ordonné_par_corpus += [c_lr_ordonné]

===DÉCÉNNIE=== 1
   [(('k', 'm'), 4862.17958257203), (('m', 't'), 4344.397098241381), (('h', 'k'), 3123.2581527287093), (('hans', 'nåde'), 2984.1378901637336), (('ma', 't'), 2638.9714342289317), (('anf', 'arb'), 1779.1857736267502), (('kon', 'ge'), 1710.1278863096165), (('hans', 'nade'), 1697.6726508316783), (('s', 'registratur'), 1356.1910188635861), (('rikets', 'råd'), 1281.9024819147244)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 1742.93324006075), (('k', 'gustafs'), 1385.7889723547237), (('kong', 'e'), 1347.846766005545), (('ma', 't'), 1260.0232167883069), (('högbe', 'te'), 1212.9883786689497), (('gustafs', 'bref'), 1173.2563167284861), (('m', 't'), 1073.8223953426295), (('h', 'k'), 876.9810075005362), (('bref', 'till'), 758.3858902441971), (('hans', 'kong'), 677.8645241628712)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 4679.59687870739), (('kong', 'e'), 4398.012843761489), (('h', 'k'), 3197.7149982837836), (('m', 't'), 2453.139965091086), (('ma', 't'), 2050.2640017170415), (('e', 'ma'), 2024.3617

Les bigrammes avec la séquence qu'on recherche ordonnés par rapports de vraisemblance

In [41]:
for i, corpus in enumerate(c_lr_ordonné_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('acta', 'publica'), 35.93075251055389), (('publica', 'från'), 23.579158633274346), (('publicis', 'comitiis'), 21.29720270260355), (('utgifna', 'publikationer'), 18.52465430341674), (('historiska', 'publikationer'), 15.842249204997778), (('större', 'publikation'), 13.262664937256957), (('dessa', 'publikationer'), 12.38563913605153), (('svenska', 'publikationer'), 11.750901938762922), (('publikation', 'som'), 11.63744037548895), (('publikationer', 'blef'), 10.698085638488465), (('publikationer', 'denna'), 10.293043885771112), (('in', 'publicis'), 9.82628717188092), (('item', 'publica'), 9.60399938413586), (('den', 'publikation'), 6.246751688627867), (('en', 'publikation'), 5.88848891734564), (('publikationer', 'af'), 5.208592278395372), (('publikation', 'af'), 5.208592278395372)]
===DÉCÉNNIE=== 2
[(('publica', 'på'), 7.0548088418745465), (('publica', 'som'), 5.9350851359449734)]
===DÉCÉNNIE=== 3
[(('publicis', 'legibus'), 21.320980010875125), (('publica', 'mercatura')